<a href="https://colab.research.google.com/github/mahsadadkhah/algorithms/blob/master/Copy_of_test_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hazm
!pip install transformers

In [ ]:
!pip install hazm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from hazm import *

from sklearn.model_selection import train_test_split


In [ ]:
# در اینجا فایل زیپ رو قرار بده و آنزیپش کن
!unzip /content/archive_5_085351.zip

Archive:  /content/archive_5_085351.zip
replace Snappfood - Sentiment Analysis.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#Read Data
df= pd.read_csv('/content/Snappfood - Sentiment Analysis.csv', sep='\t' ,encoding='utf-8', error_bad_lines=False )
df.columns

<ipython-input-5-556b4e85f24c>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df= pd.read_csv('/content/Snappfood - Sentiment Analysis.csv', sep='\t' ,encoding='utf-8', error_bad_lines=False )


Index(['Unnamed: 0', 'comment', 'label', 'label_id'], dtype='object')

In [ ]:
# حذف یک ستون اضافی
df.drop('Unnamed: 0' , axis= 1, inplace=True)

In [ ]:
df

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,عالی بود همه چه درست و به اندازه و کیفیت خوب، ...,HAPPY,0.0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...
4995,این پیتزا اصلا خوشمزه نبود,SAD,1.0
4996,تعداد اسلایسایی که فرستاده شد از تعدادی که تو ...,HAPPY,0.0
4997,برای ما قرار بود دونات شکلاتی بیارن ولی نداشتن...,HAPPY,0.0
4998,در عکس آب‌گوشت و مخلفات داشت که نبود ولی کیفیت...,HAPPY,0.0


In [ ]:
df = df. head(5000)

In [ ]:
# حذف سطر های خالی
df.label_id.unique()
df= df.dropna()

In [ ]:
# دوتا مقدار داره و مقدار خالی پاک شده
df.label_id.unique()

array([1., 0.])

In [ ]:
# نورمالیز داده ها برای اینکه برای شبکه قابل فهم تر باشه

import pandas as pd

normalizer = Normalizer()
df['comment'] = df['comment'].apply(normalizer.normalize)

In [ ]:
# تقسیم داده ها به دو ببخش تست و ترین

y= df.label_id.values.astype(int)
x= df.comment.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state= 21)

In [ ]:
!pip install transformers
import transformers

In [ ]:
# استفاده از این کلاس برای خواندن داده ها به شکلی که برای شبکه مناسب باشه

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset

# Tokenization and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,  # Adjust as needed
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }


In [ ]:
# حداکثر بچ سایز 16 می باشد

# Prepare training data
dataset = TextDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Prepare testing data
dataset2 = TextDataset(x_test, y_test)
dataloader_test = DataLoader(dataset2, batch_size=16)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# اگر خواستی تست کنی که یک بج ( شامل یک مقدار 32 تایی) رو درست خونده

# batch=next(iter(dataloader))

In [ ]:
# آموزش شبکه
loss =1
num_epochs = 25
for epoch in range(num_epochs):
  if loss < 0.3:
    break
  model.train()

  total_loss = 0
  correct = 0
  total = 0

  for i, batch in enumerate(dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

    probabilities = torch.softmax(outputs.logits, dim=1)[0]
    predicted_label = torch.argmax(probabilities).item()
    print(predicted_label)

    total += labels.size(0)
    correct += (predicted_label == labels).sum().item()

    print(f'Batch : {i} loss = {loss.item()}')

    # accuracy = 100 * correct / total

    # print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}, accuracy: {accuracy}")


0
Batch : 0 loss = 0.6804710030555725
0
Batch : 1 loss = 0.6844934225082397
1
Batch : 2 loss = 0.6685056090354919
1
Batch : 3 loss = 0.7397177815437317
1
Batch : 4 loss = 0.6926943063735962
1
Batch : 5 loss = 0.7287580370903015
0
Batch : 6 loss = 0.6976159811019897
0
Batch : 7 loss = 0.6792181730270386
0
Batch : 8 loss = 0.7218736410140991
0
Batch : 9 loss = 0.6925139427185059
0
Batch : 10 loss = 0.711522102355957
0
Batch : 11 loss = 0.6982159614562988
0
Batch : 12 loss = 0.714117169380188
0
Batch : 13 loss = 0.7158710956573486
0
Batch : 14 loss = 0.7159267067909241
0
Batch : 15 loss = 0.6903365850448608
0
Batch : 16 loss = 0.6814595460891724
0
Batch : 17 loss = 0.7030133605003357
0
Batch : 18 loss = 0.6633245348930359
0
Batch : 19 loss = 0.7013387680053711
0
Batch : 20 loss = 0.6897647380828857
1
Batch : 21 loss = 0.6590297222137451
0
Batch : 22 loss = 0.6623992323875427
1
Batch : 23 loss = 0.683091938495636
0
Batch : 24 loss = 0.6785293817520142
1
Batch : 25 loss = 0.6871498823165894

In [ ]:
# Test the model
test_text = "خیلی عالیه "
normalized_test_text = normalizer.normalize(test_text)
encoding = tokenizer.encode_plus(
    normalized_test_text,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)


In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    probabilities = torch.softmax(outputs.logits, dim=1)[0]
    predicted_label = torch.argmax(probabilities).item()

label_mapping = {0: 'happy', 1: 'sad'}
predicted_class = label_mapping[predicted_label]
print(f"The text is about: {predicted_class}")

The text is about: happy
